In [587]:
import torch
import torchvision.transforms as transforms
import pandas as pd
import torch.nn as nn

In [628]:
train_valid_dataset = pd.read_csv('BankChurners.csv')  
#application = pd.read_csv('New_BankChurners.csv') 

In [632]:
print(train_valid_dataset1)

[[15762418 'Spain' 3 ... 128643.35 1 8]
 [15749905 'Spain' 6 ... 50213.81 1 7]
 [15600911 'France' 2 ... 3061.0 0 7]
 ...
 [15636388 'Germany' 7 ... 114603.96 0 7]
 [15688951 'Germany' 8 ... 148412.24 1 9]
 [15581229 'Germany' 1 ... 122763.95 0 4]]


In [629]:
train_valid_dataset1 = train_valid_dataset.to_numpy()
weight_tensor = torch.tensor([len(train_valid_dataset1)/(len(train_valid_dataset1)-train_valid_dataset1[:,-1].sum()), 
                              len(train_valid_dataset1)/train_valid_dataset1[:,-1].sum()]).float()

In [630]:
print(weight_tensor)

tensor([-0.1861,  0.1569])


In [617]:
train_valid_dataset = train_valid_dataset.drop(columns='CustomerId')
train_valid_dataset = train_valid_dataset.drop(columns='Geography')
res_col = train_valid_dataset['CreditLevel']
train_valid_dataset = train_valid_dataset.drop(columns='CreditLevel')

#归一化
train_valid_dataset = (train_valid_dataset - train_valid_dataset.min()) / (train_valid_dataset.max() - train_valid_dataset.min())
train_valid_dataset['CreditLevel'] = res_col - 1


print(train_valid_dataset)

      Tenure   Balance  NumOfProducts  HasCrCard  IsActiveMember  \
0        0.3  0.484985       0.000000        1.0             0.0   
1        0.6  0.000000       0.000000        1.0             0.0   
2        0.2  0.728934       0.000000        1.0             0.0   
3        0.2  0.407651       0.333333        1.0             0.0   
4        0.7  0.435819       0.333333        1.0             0.0   
...      ...       ...            ...        ...             ...   
8995     0.3  0.000000       0.000000        1.0             1.0   
8996     0.6  0.000000       0.333333        1.0             1.0   
8997     0.7  0.393687       0.000000        1.0             0.0   
8998     0.8  0.476905       0.333333        0.0             1.0   
8999     0.1  0.690881       0.000000        0.0             1.0   

      EstimatedSalary  Exited  CreditLevel  
0            0.643290     1.0            7  
1            0.251062     1.0            6  
2            0.015250     0.0            6  
3  

In [618]:
valid_ratio = 0.2 

#weight_tensor = torch.tensor([len(train_valid_dataset)/(len(train_valid_dataset)-train_valid_dataset[:,-1].sum()), len(train_valid_dataset)/train_valid_dataset[:,-1].sum()]).float() 

nb_train = int((1.0 - valid_ratio) * len(train_valid_dataset))
nb_valid =  int(valid_ratio * len(train_valid_dataset))
train_dataset, valid_dataset = torch.utils.data.dataset.random_split(train_valid_dataset.to_numpy(), [nb_train, nb_valid])

In [619]:
class DatasetTransformer(torch.utils.data.Dataset):

    def __init__(self, base_dataset, transform=transforms.Lambda(lambda x: x)):
        self.base_dataset = base_dataset
        self.transform = transform

    def __getitem__(self, index):
        inpt, target = torch.from_numpy(self.base_dataset[index][:-1]), self.base_dataset[index][-1]
        return self.transform(inpt).float(), int(target)

    def __len__(self):
        return len(self.base_dataset)


train_dataset = DatasetTransformer(train_dataset)
valid_dataset = DatasetTransformer(valid_dataset)

In [620]:
#Dataloader

batch_size  = 100   # Using minibatches of X samples

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

valid_loader = torch.utils.data.DataLoader(dataset=valid_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)


print("The train set contains {} samples, in {} batches".format(len(train_loader.dataset), len(train_loader)))
print("The validation set contains {} samples, in {} batches".format(len(valid_loader.dataset), len(valid_loader)))

The train set contains 7200 samples, in 72 batches
The validation set contains 1800 samples, in 18 batches


In [621]:
#Define device
use_gpu = torch.cuda.is_available()
if use_gpu:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [622]:
def linear_relu(dim_in, dim_out):
    return [nn.Linear(dim_in, dim_out),
            nn.ReLU(inplace=True)]

class FullyConnected(nn.Module):

    def __init__(self, input_size, num_classes):
        super(FullyConnected, self).__init__()
        self.classifier =  nn.Sequential(
            #nn.Dropout(0.2),
            *linear_relu(input_size, 14),
            #nn.Dropout(0.5), #Generally 0.2 for the input layer and 0.5 for the hidden layer
            *linear_relu(14, 28),
            #nn.Dropout(0.5),
            nn.Linear(28, num_classes)
        )

    def forward(self, x):
        x = x.view(x.size()[0], -1)
        y = self.classifier(x)
        return y


model = FullyConnected(7, 10)
model.to(device)

FullyConnected(
  (classifier): Sequential(
    (0): Linear(in_features=7, out_features=14, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=14, out_features=28, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=28, out_features=10, bias=True)
  )
)

In [623]:
def train(model, loader, f_loss, optimizer, device):
    """
    Train a model for one epoch, iterating over the loader
    using the f_loss to compute the loss and the optimizer
    to update the parameters of the model.

    Arguments :

        model     -- A torch.nn.Module object
        loader    -- A torch.utils.data.DataLoader
        f_loss    -- The loss function, i.e. a loss Module
        optimizer -- A torch.optim.Optimzer object
        device    -- a torch.device class specifying the device
                     used for computation

    Returns :
    """

    # We enter train mode. This is useless for the linear model
    # but is important for layers such as dropout, batchnorm, ...
    model.train()

    for i, (inputs, targets) in enumerate(loader):
        
        
        inputs, targets = inputs.to(device), targets.to(device)
        # Compute the forward pass through the network up to the loss
        outputs = model(inputs)
        
        #print(targets)
        #print(outputs)
        
        loss = f_loss(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
def test(model, loader, f_loss, device):
    """
    Test a model by iterating over the loader

    Arguments :

        model     -- A torch.nn.Module object
        loader    -- A torch.utils.data.DataLoader
        f_loss    -- The loss function, i.e. a loss Module
        device    -- The device to use for computation 

    Returns :

        A tuple with the mean loss, mean accuracy and mean unbiaised accuracy

    """
    # We disable gradient computation which speeds up the computation
    # and reduces the memory usage
    with torch.no_grad():
        # We enter evaluation mode. This is useless for the linear model
        # but is important with layers such as dropout, batchnorm, ..
        model.eval()
        N = 0
        tot_loss, correct, unbiaised_acc = 0.0, 0.0, 0.0
        for i, (inputs, targets) in enumerate(loader):

            # We got a minibatch from the loader within inputs and targets

            # We need to copy the data on the GPU if we use one
            inputs, targets = inputs.to(device), targets.to(device)

            # Compute the forward pass, i.e. the scores for each input
            outputs = model(inputs)

            # We accumulate the exact number of processed samples
            N += inputs.shape[0]

            # We accumulate the loss considering
            # The multipliation by inputs.shape[0] is due to the fact
            # that our loss criterion is averaging over its samples
            tot_loss += inputs.shape[0] * f_loss(outputs, targets).item()

            # For the accuracy, we compute the labels for each input
            # Be carefull, the model is outputing scores and not the probabilities
            # But given the softmax is not altering the rank of its input scores
            # we can compute the label by argmaxing directly the scores
            predicted_targets = outputs.argmax(dim=1)
            
            correct += (predicted_targets == targets).sum().item()
            
            #Compute the unbiaised accuracy
            for value in predicted_targets.unique() :
                mask = (predicted_targets == targets) & (predicted_targets == value.item())
                
        return tot_loss/N, correct/N, unbiaised_acc/N

In [624]:
class ModelCheckpoint:

    def __init__(self, filepath, model):
        self.min_loss = None
        self.filepath = filepath
        self.model = model

    def update(self, loss):
        if (self.min_loss is None) or (loss < self.min_loss):
            print("Saving a better model")
            torch.save(self.model.state_dict(), self.filepath)
            self.min_loss = loss
            
            
model_path = "best_model.pt"

In [625]:
print(len(train_loader))

72


In [626]:
epochs = 50
optimizer = torch.optim.Adam(model.parameters())
f_loss = torch.nn.CrossEntropyLoss()
f_loss = torch.nn.CrossEntropyLoss(weight=weight_tensor.to(device))
model_checkpoint = ModelCheckpoint(model_path, model)

for t in range(epochs):
    print("\nEpoch {}".format(t))
    train(model, train_loader, f_loss, optimizer, device)
    train_loss, train_acc, train_unb_acc = test(model, train_loader, f_loss, device)
    print(" Train : Loss : {:.4f}, Acc : {:.4f}, Unb.Acc. : {:.4f}".format(train_loss, train_acc, train_unb_acc))

    val_loss, val_acc, val_unb_acc = test(model, valid_loader, f_loss, device)
    print(" Validation : Loss : {:.4f}, Acc : {:.4f}, Unb.Acc. : {:.4f}".format(val_loss, val_acc, val_unb_acc))

    model_checkpoint.update(val_loss)


model.load_state_dict(torch.load(model_path))

# Switch to eval mode 
model.eval()

test_loss, test_acc, test_unb_acc = test(model, valid_loader, f_loss, device)
print("\n\n Test : Loss : {:.4f}, Acc. : {:.4f}, Unb.Acc. : {:.4f}".format(test_loss, test_acc, test_unb_acc))


Epoch 0


RuntimeError: weight tensor should be defined either for all 10 classes or no classes but got weight tensor of shape: [2]